# GP-CEI

In [1]:
import torch
import numpy as np

from botorch.models import SingleTaskGP
from gpytorch.mlls.exact_marginal_log_likelihood import ExactMarginalLogLikelihood
from botorch.fit import fit_gpytorch_mll
from botorch.acquisition.analytic import ConstrainedExpectedImprovement
from botorch.optim import optimize_acqf

from GP_CEI import get_and_fit_gp, get_next_candidates_CEI

# Select your test case
from test_functions.Ackley2D import Ackley2D, Ackley2D_Scaling
from constraint_handling.PenaltyFunction import PenaltyFunction

import warnings
warnings.filterwarnings('ignore')


## Bayesian Optimization 

In [2]:
# Set up constants
N_iter = 50                  # Iteration number
rho = 1                      # Initial rho for penalty function 
rho_scaling_factor = 1.5
Current_BEST = -1e10         # Some arbitrary very small number
Prev_BEST = -1e10 
COUNT = 0                    # Count for updating the scaling factor



# Get initial random samples
trained_X = torch.rand(20,2)   # Initial samples (20 samples)

# Scale it to the domain of interest
X_Scaled = Ackley2D_Scaling(trained_X)

# Get the constraints and objective
trained_gx,trained_Y = Ackley2D(X_Scaled)

In [3]:


# BO loop

for ii in range(N_iter):

    
    # (0) Get the updated data for this iteration
    X_scaled = Ackley2D_Scaling(trained_X)
    trained_gx, trained_Y = Ackley2D(X_scaled)


    
    # (1) Confirm the best so far
    if ((trained_gx<=0).all(dim=1)).any():
        Current_BEST = torch.max(trained_Y[(trained_gx<=0).all(dim=1)])
    else:
            Current_BEST = Prev_BEST

    if Current_BEST <= Prev_BEST:
            COUNT += 1     
    if COUNT >= 5:
        rho = rho*rho_scaling_factor # Updating the rho for penalty function 
        COUNT = 0
    Prev_BEST = Current_BEST 

    # (2) Fit the Gaussian Process and Calculate CEI
    best_candidate = get_next_candidates_CEI(trained_X, trained_gx, trained_Y, Prev_BEST)
    
    # (3) Append the next search point
    trained_X = torch.cat([trained_X, best_candidate])


print( 'Best feasible value: {v}'.format(v=Current_BEST) )


Best feasible value: -0.7679738998413086
